# DeepLandforms Training YOLOv5
## Landforms object detection training
@ g.nodjoumi@jacobs-university.de

***This notebook is based on yolov5 repository is provided by ultralytics https://zenodo.org/record/4154370 that has been modified to adapt to landforms***

## Dataset
If you are using Roboflow prepared dataset skip this part, else prepare the dataset folder as follow:

Organize the dataset folder as following:

* *datasetfolder/train/images*
* *datasetfolder/valid/images*

then create a dataset.yaml file containing:

* train: *path to datasetfolder/train/images*
* valid: *path to datasetfolder/valid/images*
* nc: *number of classes*
* names: ['label1','label2','...']

## How-TO
Just run the notebook and it will ask all variables and parameters necessary for training.

*To further personalize the training, edit the dictionary, in the dedicated cell below, before run the notebook*

In [1]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
import shutil
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

fatal: il path di destinazione 'yolov5' esiste già e non è una directory vuota.
/home/hyradus/MyDrive/PyS/DeepLandforms-YOLOv5/yolov5
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Setup complete. Using torch 1.10.2+cu102 (NVIDIA GeForce GTX 1080 Ti)


### Import modules

In [3]:
try:
    shutil.copytree('../adds/','../yolov5/adds')    
except Exception as e:
    print(e)
from adds.GenUtils import question, askPath, askFile, askInt
from adds.DLUtils import get_train_cfg, link_dataset, get_model_cfg, get_args

[Errno 17] File exists: '../yolov5/adds'


In [4]:
print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 1.10.2+cu102 _CudaDeviceProperties(name='NVIDIA GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11175MB, multi_processor_count=28)


### Set paths

In [21]:
home = os.getcwd()
logdir = os.path.dirname(home)+'/train_logs'
os.makedirs(logdir, exist_ok=True)
models_path = home+'/models'
source = askPath('Dataset folder')
yaml = source+'/data.yaml'

Insert path to Dataset folder /mnt/DATA/Working/DeepLandforms-YOLOv5/Dataset/


/mnt/DATA/Working/DeepLandforms-YOLOv5/Dataset/ is valid


In [22]:
%cat $yaml

train: train/images
val: valid/images

nc: 5
names: ['1', '2', '3', '4', '5']


### Get model configuration

In [24]:
model = int(question('Please select 1: yolov5s, 2:yolov5m, 3: yolov5l, 4:yolov5x', ['1','2','3','4']))
train_cfg, model_name, run_folder = get_train_cfg(logdir, yaml, models_path, model)

Please enter only: 
1, 2, 3, 4



Please select 1: yolov5s, 2:yolov5m, 3: yolov5l, 4:yolov5x  Answer:  1


03-03-2022_23-53-43  Folder not exist, creating.
Created new  03-03-2022_23-53-43  Folder


### Create symlink between repository and data folder

In [25]:
try:
    link_dataset(home,source)
except Exception as e:
    print(e)
    pass

[Errno 17] File exists: '/mnt/DATA/Working/DeepLandforms-YOLOv5/Dataset//train' -> '/home/hyradus/MyDrive/PyS/DeepLandforms-YOLOv5/yolov5/train'


### Set training parameters

In [26]:
epochs = askInt('Epochs: ')
img_size = askInt('Image size: ')
batch_size = askInt('Batch size: ')

Insert integer for Epochs:  500
Insert integer for Image size:  640
Insert integer for Batch size:  8


### Create dictionary with all parameters
***Edit items if needed***

In [15]:
args_dict = {
    '--epochs':epochs, #integer
    '--batch-size':batch_size, #integer
    '--weights': model_name+'.pt',#source+'/trained_weights', #path
    '--cfg':train_cfg, #path
 	'--data':yaml, #path
 	'--hyp':'', #path
 	'--img-size':img_size, #integer
 	'--rect':'', #True of False, '' = False
 	'--resume':'', #True of False, '' = False
 	'--nosave':'', #True of False, '' = False
 	'--notest':'', #True of False, '' = False
 	'--noautoanchor':'', #True of False, '' = False
 	'--evolve':'', #True of False, '' = False
 	'--bucket':'', #True of False, '' = False
 	#'--cache-images':True, #True of False, '' = False
 	'--image-weights':'', #True of False, '' = False
 	'--device':'', #cuda or cpu, auto filled
 	'--multi-scale':'', #True of False, '' = False
 	'--single-cls':'', #True of False, '' = False
 	'--adam':'',  #True of False, '' = False
 	'--sync-bn':'', #True of False, '' = False
    '--local_rank':'', #int
    '--log-imgs':'', # int
    '--log-artifacts':'', #True of False, '' = False
 	'--workers':'', #integer
    '--project':logdir, #path
    '--name': run_folder, #string
    '--exist-ok':'', #True of False, '' = False
    '--quad':''} #True of False, '' = False

### Get arguments for training script

In [16]:
arv = get_args(args_dict)
arv

'--epochs 10 --batch-size 8 --weights yolov5x.pt --cfg /home/hyradus/MyDrive/PyS/DeepLandforms-YOLOv5/train_logs/03-03-2022_16-31-48/custom_yolov5x.yaml --data /home/hyradus/MyDrive/PyS/DeepLandforms-YOLOv5/Dataset/data.yaml --img-size 640 --project /home/hyradus/MyDrive/PyS/DeepLandforms-YOLOv5/train_logs --name 03-03-2022_16-31-48'

### Training

In [17]:
%load_ext tensorboard
%tensorboard --logdir $logdir

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 134027), started 0:01:00 ago. (Use '!kill 134027' to kill it.)

In [18]:
os.chdir(home)
clear_output
!python train.py $arv

train: weights=yolov5x.pt, cfg=/home/hyradus/MyDrive/PyS/DeepLandforms-YOLOv5/train_logs/03-03-2022_16-31-48/custom_yolov5x.yaml, data=/home/hyradus/MyDrive/PyS/DeepLandforms-YOLOv5/Dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/home/hyradus/MyDrive/PyS/DeepLandforms-YOLOv5/train_logs, name=03-03-2022_16-31-48, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-11-g63ddb6f torch 1.10.2+cu102 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11176MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.